**Creating & Cleaning the Full Dataset: 2011-2025**
---------------
This script takes in datasets which have been manually manipulated (ensuring each has matched column names, a task easier done by hand than by programming each case one-at-a-time) and processes them into one extended dataset.

This dataset is reduced from the more extensive Oct 2015 - Sept 2023 datasets, including near-exclusively the data which is *also* contained in the recent 2023-25 FOIA release. Additions to the dataset that are not present post-2023 are ethnicity, race, and "worst criminal charge" (which refers to the most serious criminal charge -- conviction notwithstanding -- of the apprehended individual)

As less than 5% of the 2023-2025 dataset is labeled as "duplicate likely" (<0.5% of the entire dataset), all marked potential duplicates are dropped to provide a **minimal estimate** for all variables after 2023.

The extended dataset is 1,695,332 entries long after discarding ~6000 potential duplicates.

In [151]:
import numpy as np
import pandas as pd
import glob, os, warnings, datetime

path_to_files = "Desktop/arrests/"
output_filename = "Arrests 2011-2025 (clean)"

#Thanks Trenton McKinney on stackoverflow for this clean little snippet to mass-import & concat csv!
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
all_files = glob.glob(os.path.join(path_to_files, "*.csv"))
a12_24 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

#drop the highly year-specific & redundant cols, format variable headers 
a12_24 = a12_24.drop(['unique_identifier', 'sheet_original', 'row_original', 'final_order_date', 'case_status', 'apprehension_date', 'case_category', 'file_original', 'final_order_yes_no'], axis=1)
a12_24.columns = [col.replace("_", " ").title() for col in a12_24.columns]
a12_24 = a12_24.rename(columns={"Apprehension Aor":"Apprehension AOR", "Final Order Yes No":"Final Order"})

#silent downcasting is deprecated but effective
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    duplicate_indices = a12_24[a12_24["Duplicate Likely"].fillna(False)].index
    a12_24 = a12_24.drop(duplicate_indices).drop('Duplicate Likely', axis=1)

In [ ]:
#parses inconsistent date_time data. takes str as obj, returns datetime obj. wicked slow, but functional
def date_time(moment):
    moment = str(moment)
    #split into YYYY, MM, DD, HH:MM:SS or MM, DD, YYYY, HH:MM:SS
    pieces = moment.replace(" UTC", "").replace("/","-").strip().replace(" ", "-").split("-")

    #figure out which of the two formats this is
    year_ind = 2 if (len(pieces[0]) != 4) else  0

    #separate year, normalize formats
    year = int(pieces[year_ind])
    pieces.pop(year_ind)
    month = int(pieces[0])
    day = int(pieces[1])

    #handle blank timestamps, save to datetime obj & return
    try:
        hours, minutes, seconds = pieces[2].split(":")
        return datetime.datetime(year, month, day, int(hours), int(minutes), int(seconds))
    except IndexError:
        return datetime.datetime(year, month, day)

#extract year for readibility outside of python scripts & ease of handling within
a12_24["Apprehension Year"] = a12_24["Apprehension Date Time"].str.extract('([0-9]{4})')

#normalize into datetime objects
a12_24["Apprehension Date Time"].transform(date_time)

a12_24 = a12_24.reindex(sorted(a12_24.columns), axis=1)
a12_24.to_csv(path_to_files, output_filename, '.csv') #save a copy

**Some variables are not tracked across the entire timespan**
---------------
**Apprehension Criminality & State, Birth Year, Departure Country, Race, Ethnicity, Final Program, Final Program Group**, and **Worst Criminal Charge** all start being tracked in 2015. **Ethnicity, Race,** and **Worst Criminal Charge** also lack data following 2023.

So, here's a handy-dandy chart to visualize the span of each variable in the complete dataset!


In [152]:
vars_by_year = a12_24.groupby("Apprehension Year").count().astype(bool) #true if variable is recorded for a given year
vars_by_year = vars_by_year.transpose()

def colour_truth_false(val):
    color = 'blue' if val else 'red'
    return 'color: %s' %color

vars_by_year.style.map(colour_truth_false)